In [ ]:
#hide
from nbdev.showdoc import show_doc

# 10-fold Cross Validation for Augmented Digits Data
> Code for training is the same as Notebook 05.

In [ ]:
from car_speech.fname_processing import load_fnames
from car_speech.pipeline import *

import string
from random import shuffle
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.model_selection import train_test_split, KFold

## Configuration

In [ ]:
DATASET_TYPE = 'digits' # 'digits' or 'letters' or 'mixed'
LEVEL_NAME = '35U' # 'IDL' or '35U' or '35D' or '55U' or '55D'

EPOCHS = 100

In [ ]:
label_strings = np.array([str(num) for num in range(0,10)])

# load classified filenames
filenames = load_fnames('noise_levels/digit_augmented_noise_levels/' + LEVEL_NAME + '.data')
num_samples = len(filenames)
print('number of files:', len(filenames))

# shuffle
shuffle(filenames)

fold_no = 1
acc_per_fold = []

number of files: 6360


## Loop for 10-fold

In [ ]:
beg_ratio = 0
while beg_ratio < 0.9:
    test_beg = int(beg_ratio*num_samples)
    test_end = int((beg_ratio+0.1)*num_samples)
    
    fnames_copy = deepcopy(filenames)
    test_files = fnames_copy[test_beg:test_end]
    del fnames_copy[test_beg:test_end]
    
    val_len = len(test_files)
    val_files = fnames_copy[-val_len:]
    del fnames_copy[-val_len:]
    
    train_files = fnames_copy
    
    beg_ratio += 0.1
    
    train_files = shuffle_data(train_files)
    val_files = shuffle_data(val_files)
    test_files = shuffle_data(test_files)
    

    # Process data using the combined pipeline
    spectrogram_ds = preprocess_dataset(train_files, DATASET_TYPE)
    train_ds = spectrogram_ds
    val_ds = preprocess_dataset(val_files, DATASET_TYPE)
    test_ds = preprocess_dataset(test_files, DATASET_TYPE)
    print("Pipeline Completed")
    
    # batch
    batch_size = 64
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)
    
    # model
    for spectrogram, _ in spectrogram_ds.take(1):
        input_shape = spectrogram.shape
#     print('Input shape:', input_shape)
    num_labels = len(label_strings)

    norm_layer = preprocessing.Normalization()
    norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

#     model = models.Sequential([
#         layers.Input(shape=input_shape),
#         preprocessing.Resizing(32, 32), 
#         norm_layer,
#         layers.Conv2D(32, 3, activation='relu'),
#         layers.Conv2D(64, 3, activation='relu'),
#         layers.MaxPooling2D(),
#         layers.Dropout(0.25),
#         layers.Flatten(),
#         layers.Dense(128, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(num_labels),
#     ])

    model = models.Sequential([
        layers.Input(shape=input_shape),
        preprocessing.Resizing(32, 32), 
        norm_layer,
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_labels),
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    
    # Train
    history = model.fit(
        train_ds, 
        validation_data=val_ds,  
        epochs=EPOCHS,
        callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
    )
    
    # Test
    test_audio = []
    test_labels = []

    for audio, label in test_ds:
        test_audio.append(audio.numpy())
        test_labels.append(label.numpy())

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)
    
    y_pred = np.argmax(model.predict(test_audio), axis=1)
    y_true = test_labels

    test_acc = sum(y_pred == y_true) / len(y_true)
    print(f'Fold {fold_no} - Test set accuracy: {test_acc:.0%}')
    acc_per_fold.append(round(test_acc, 2))
    fold_no += 1

print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Pipeline Completed
Epoch 1/100
80/80 [==============================] - 32s 398ms/step - loss: 2.1771 - accuracy: 0.1865 - val_loss: 1.9490 - val_accuracy: 0.2846
Epoch 2/100
80/80 [==============================] - 10s 120ms/step - loss: 1.8984 - accuracy: 0.3031 - val_loss: 1.6944 - val_accuracy: 0.3758
Epoch 3/100
80/80 [==============================] - 9s 118ms/step - loss: 1.6860 - accuracy: 0.3748 - val_loss: 1.4818 - val_accuracy: 0.4670
Epoch 4/100
80/80 [==============================] - 9s 118ms/step - loss: 1.5420 - accuracy: 0.4336 - val_loss: 1.3308 - val_accuracy: 0.5236
Epoch 5/100
80/80 [==============================] - 9s 118ms/step - loss: 1.4205 - accuracy: 0.4760 - val_loss: 1.2181 - val_accuracy: 0.5613
Epoch 6/100
80/80 [==============================] - 9s 117ms/step - loss: 1.3412 - accuracy: 0.5161 - val_loss: 1.1225 - val_accuracy: 0.5943
Epoch 7/100
80/80 [==============================] - 9s 118ms/step - loss: 1.2611 - accuracy: 0.5407 - val_loss: 1.0504 -

80/80 [==============================] - 10s 126ms/step - loss: 0.7094 - accuracy: 0.7362 - val_loss: 0.7120 - val_accuracy: 0.7374
Epoch 33/100
80/80 [==============================] - 10s 130ms/step - loss: 0.7034 - accuracy: 0.7406 - val_loss: 0.7002 - val_accuracy: 0.7374
Epoch 34/100
80/80 [==============================] - 13s 158ms/step - loss: 0.7057 - accuracy: 0.7435 - val_loss: 0.6856 - val_accuracy: 0.7500
Epoch 35/100
80/80 [==============================] - 18s 227ms/step - loss: 0.7020 - accuracy: 0.7410 - val_loss: 0.7070 - val_accuracy: 0.7248
Epoch 36/100
80/80 [==============================] - 16s 194ms/step - loss: 0.6777 - accuracy: 0.7522 - val_loss: 0.6991 - val_accuracy: 0.7343
Epoch 37/100
80/80 [==============================] - 13s 157ms/step - loss: 0.6893 - accuracy: 0.7526 - val_loss: 0.7043 - val_accuracy: 0.7390
Epoch 00037: early stopping
Fold 2 - Test set accuracy: 70%
Pipeline Completed
Epoch 1/100
80/80 [==============================] - 30s 372ms/s

80/80 [==============================] - 13s 157ms/step - loss: 1.4729 - accuracy: 0.4475 - val_loss: 1.2523 - val_accuracy: 0.5613
Epoch 6/100
80/80 [==============================] - 13s 157ms/step - loss: 1.3751 - accuracy: 0.4959 - val_loss: 1.1759 - val_accuracy: 0.5833
Epoch 7/100
80/80 [==============================] - 11s 141ms/step - loss: 1.2741 - accuracy: 0.5356 - val_loss: 1.0716 - val_accuracy: 0.6101
Epoch 8/100
80/80 [==============================] - 12s 150ms/step - loss: 1.2165 - accuracy: 0.5507 - val_loss: 1.0320 - val_accuracy: 0.6305
Epoch 9/100
80/80 [==============================] - 12s 153ms/step - loss: 1.1617 - accuracy: 0.5704 - val_loss: 0.9840 - val_accuracy: 0.6494
Epoch 10/100
80/80 [==============================] - 12s 151ms/step - loss: 1.0886 - accuracy: 0.6030 - val_loss: 0.9123 - val_accuracy: 0.6714
Epoch 11/100
80/80 [==============================] - 11s 141ms/step - loss: 1.0520 - accuracy: 0.6085 - val_loss: 0.9140 - val_accuracy: 0.6730
Ep

80/80 [==============================] - 11s 143ms/step - loss: 1.3618 - accuracy: 0.5028 - val_loss: 1.1931 - val_accuracy: 0.5723
Epoch 7/100
80/80 [==============================] - 11s 138ms/step - loss: 1.2767 - accuracy: 0.5332 - val_loss: 1.0651 - val_accuracy: 0.6164
Epoch 8/100
80/80 [==============================] - 11s 139ms/step - loss: 1.2249 - accuracy: 0.5586 - val_loss: 1.0320 - val_accuracy: 0.6447
Epoch 9/100
80/80 [==============================] - 12s 148ms/step - loss: 1.1435 - accuracy: 0.5886 - val_loss: 0.9409 - val_accuracy: 0.6541
Epoch 10/100
80/80 [==============================] - 12s 154ms/step - loss: 1.1181 - accuracy: 0.5916 - val_loss: 0.9068 - val_accuracy: 0.6651
Epoch 11/100
80/80 [==============================] - 12s 144ms/step - loss: 1.0542 - accuracy: 0.6187 - val_loss: 0.8812 - val_accuracy: 0.6667
Epoch 12/100
80/80 [==============================] - 11s 143ms/step - loss: 1.0147 - accuracy: 0.6321 - val_loss: 0.8629 - val_accuracy: 0.6667
E

80/80 [==============================] - 10s 119ms/step - loss: 1.7050 - accuracy: 0.3686 - val_loss: 1.6211 - val_accuracy: 0.4000
Epoch 4/100
80/80 [==============================] - 10s 122ms/step - loss: 1.5985 - accuracy: 0.4092 - val_loss: 1.4575 - val_accuracy: 0.4425
Epoch 5/100
80/80 [==============================] - 10s 121ms/step - loss: 1.4803 - accuracy: 0.4562 - val_loss: 1.3119 - val_accuracy: 0.5008
Epoch 6/100
80/80 [==============================] - 10s 120ms/step - loss: 1.3794 - accuracy: 0.4996 - val_loss: 1.1967 - val_accuracy: 0.5638
Epoch 7/100
80/80 [==============================] - 10s 127ms/step - loss: 1.2923 - accuracy: 0.5344 - val_loss: 1.1075 - val_accuracy: 0.5969
Epoch 8/100
80/80 [==============================] - 11s 133ms/step - loss: 1.2135 - accuracy: 0.5538 - val_loss: 1.1048 - val_accuracy: 0.5858
Epoch 9/100
80/80 [==============================] - 10s 122ms/step - loss: 1.1599 - accuracy: 0.5737 - val_loss: 1.0424 - val_accuracy: 0.6126
Epoc

80/80 [==============================] - 9s 118ms/step - loss: 0.8184 - accuracy: 0.6969 - val_loss: 0.8864 - val_accuracy: 0.6950
Epoch 23/100
80/80 [==============================] - 10s 119ms/step - loss: 0.8136 - accuracy: 0.7018 - val_loss: 0.8425 - val_accuracy: 0.6965
Epoch 24/100
80/80 [==============================] - 10s 120ms/step - loss: 0.8159 - accuracy: 0.7026 - val_loss: 0.8361 - val_accuracy: 0.7060
Epoch 25/100
80/80 [==============================] - 9s 117ms/step - loss: 0.7855 - accuracy: 0.7077 - val_loss: 0.8409 - val_accuracy: 0.6855
Epoch 26/100
80/80 [==============================] - 9s 119ms/step - loss: 0.7746 - accuracy: 0.7201 - val_loss: 0.8401 - val_accuracy: 0.7107
Epoch 27/100
80/80 [==============================] - 9s 119ms/step - loss: 0.7674 - accuracy: 0.7164 - val_loss: 0.8334 - val_accuracy: 0.7013
Epoch 28/100
80/80 [==============================] - 10s 121ms/step - loss: 0.7460 - accuracy: 0.7254 - val_loss: 0.8515 - val_accuracy: 0.7013
Ep

In [ ]:
print("Done")
print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Done
[0.68, 0.7, 0.7, 0.68, 0.69, 0.7, 0.72, 0.68, 0.69, 0.75]
mean score: 0.7
